In [16]:
import sqlite3
conn = sqlite3.connect("AQI1.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [15]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)
print(sitenames)

['彰化（員林）', '古亭', '萬華', '松山', '宜蘭', '永和', '忠明', '善化', '中壢', '金門', '恆春', '萬里', '彰化', '橋頭', '麥寮', '平鎮', '朴子', '斗六', '楠梓', '桃園', '大城', '馬公', '龍潭', '富貴角', '大園', '屏東(枋山)', '潮州', '仁武', '中山', '板橋', '鳳山', '埔里', '基隆', '小港', '竹山', '陽明', '線西', '土城', '新營', '士林', '前鎮', '林園', '大寮', '嘉義', '西屯', '新港', '新店', '三義', '大同', '汐止', '新莊', '冬山', '馬祖', '頭份', '臺南', '關山', '花蓮', '林口', '臺西', '竹東', '前金', '豐原', '菜寮', '大里', '沙鹿', '屏東（琉球）', '左營', '安南', '臺東', '屏東', '湖口', '復興', '淡水', '觀音', '美濃', '新竹', '三重', '新北(樹林)', '高雄（湖內）', '苗栗', '南投', '臺南（麻豆）', '崙背', '二林', '宜蘭（頭城）']
['彰化（員林）', '古亭', '萬華', '松山', '宜蘭', '永和', '忠明', '善化', '中壢', '金門', '恆春', '萬里', '彰化', '橋頭', '麥寮', '平鎮', '朴子', '斗六', '楠梓', '桃園', '大城', '馬公', '龍潭', '富貴角', '大園', '屏東(枋山)', '潮州', '仁武', '中山', '板橋', '鳳山', '埔里', '基隆', '小港', '竹山', '陽明', '線西', '土城', '新營', '士林', '前鎮', '林園', '大寮', '嘉義', '西屯', '新港', '新店', '三義', '大同', '汐止', '新莊', '冬山', '馬祖', '頭份', '臺南', '關山', '花蓮', '林口', '臺西', '竹東', '前金', '豐原', '菜寮', '大里', '沙鹿', '屏東（琉球）', '左營', '安南', '臺東', '屏東', '湖口', '復興', '淡水', '觀音', '美

In [13]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

[['2024-11-04 16:00', '屏東縣', 64, 14, '普通', 22.260899, 120.651472],
 ['2024-11-04 16:00', '屏東縣', 64, 14, '普通', 22.260899, 120.651472],
 ['2024-11-04 15:00', '屏東縣', 50, 20, '良好', 22.260899, 120.651472],
 ['2024-11-04 15:00', '屏東縣', 50, 20, '良好', 22.260899, 120.651472],
 ['2024-11-04 15:00', '屏東縣', 50, 20, '良好', 22.260899, 120.651472],
 ['2024-11-04 15:00', '屏東縣', 50, 20, '良好', 22.260899, 120.651472],
 ['2024-11-04 14:00', '屏東縣', 46, 14, '良好', 22.260899, 120.651472],
 ['2024-11-04 14:00', '屏東縣', 46, 14, '良好', 22.260899, 120.651472],
 ['2024-11-04 14:00', '屏東縣', 46, 14, '良好', 22.260899, 120.651472],
 ['2024-11-04 14:00', '屏東縣', 46, 14, '良好', 22.260899, 120.651472],
 ['2024-11-04 13:00', '屏東縣', 43, 21, '良好', 22.260899, 120.651472],
 ['2024-11-04 13:00', '屏東縣', 43, 21, '良好', 22.260899, 120.651472],
 ['2024-11-04 13:00', '屏東縣', 43, 21, '良好', 22.260899, 120.651472],
 ['2024-11-04 13:00', '屏東縣', 43, 21, '良好', 22.260899, 120.651472],
 ['2024-11-04 12:00', '屏東縣', 39, 17, '良好', 22.260899, 120.6514

In [12]:
import sqlite3
conn = sqlite3.connect("AQI1.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [11]:
import sqlite3
conn = sqlite3.connect("AQI1.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))



        

In [10]:
conn = sqlite3.connect("AQI1.db")
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT DISTINCT sitename
    FROM records
    '''
    cursor.execute(sql)
    sitenames = []

    for items in cursor.fetchall():
        sitenames.append(items[0])
    
    print(sitenames)

['屏東(枋山)', '宜蘭（頭城）', '新北(樹林)', '屏東（琉球）', '臺南（麻豆）', '高雄（湖內）', '彰化（員林）', '大城', '富貴角', '麥寮', '關山', '馬公', '金門', '馬祖', '埔里', '復興', '永和', '竹山', '中壢', '三重', '冬山', '宜蘭', '陽明', '花蓮', '臺東', '恆春', '潮州', '屏東', '小港', '前鎮', '前金', '左營', '楠梓', '林園', '大寮', '鳳山', '仁武', '橋頭', '美濃', '臺南', '安南', '善化', '新營', '嘉義', '臺西', '朴子', '新港', '崙背', '斗六', '南投', '二林', '線西', '彰化', '西屯', '忠明', '大里', '沙鹿', '豐原', '三義', '苗栗', '頭份', '新竹', '竹東', '湖口', '龍潭', '平鎮', '觀音', '大園', '桃園', '大同', '松山', '古亭', '萬華', '中山', '士林', '淡水', '林口', '菜寮', '新莊', '板橋', '土城', '新店', '萬里', '汐止', '基隆']


In [9]:
conn = sqlite3.connect("AQI1.db")
with conn:
    cursor = conn.cursor()        
    sql = '''
    SELECT date,county,aqi,pm25,status,lat,lon
    FROM records
    WHERE sitename=?
    ORDER BY date DESC;
    '''
    cursor.execute(sql,('富貴角',))
    sitename_list = [list(item) for item in cursor.fetchall()]
    print(sitename_list)

[['2024-11-04 16:00', '新北市', 41, 12, '良好', 25.29681695, 121.53656894], ['2024-11-04 15:00', '新北市', 46, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 15:00', '新北市', 46, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 14:00', '新北市', 47, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 14:00', '新北市', 0, 0, '', 25.29681695, 121.53656894], ['2024-11-04 13:00', '新北市', 47, 8, '良好', 25.29681695, 121.53656894], ['2024-11-04 13:00', '新北市', 47, 8, '良好', 25.29681695, 121.53656894], ['2024-11-04 12:00', '新北市', 48, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 12:00', '新北市', 48, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 11:00', '新北市', 48, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 11:00', '新北市', 48, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 10:00', '新北市', 48, 13, '良好', 25.29681695, 121.53656894]]
